## Table of Contents
* [Introduction](#Introduction)
* [Imports](#Imports)
* [Using LCviz](#Using-LCviz)
    * [Initializing LCviz](#Initializing-LCviz)
    * [Loading Light Curves into LCviz](#Loading-light-curves-into-LCviz)
    * [Flattening](#Flattening)
    * [Phase Folding](#Phase-Folding)
* [Additional Resources](#Additional-Resources)

## Introduction
In this notebook we will be exploring the use of the [LCviz](https://github.com/spacetelescope/lcviz) (light curve visualization and analysis) tool for investigating time series observations of a transiting exoplanet. LCviz is built on the [Jdaviz](https://github.com/spacetelescope/jdaviz/) data analysis and visualization tool and additionally relies heavily on the [lightkurve](https://github.com/lightkurve/lightkurve) and [astropy](https://github.com/astropy/astropy) packages.

Specifically, we will be using LCviz to look at a Kepler long cadence light curve of HAT-P-11, a K4V host to a transiting hot Neptune with a 4.8 day period, and a stellar rotation period of 29 days ([Morris et al. 2014](https://ui.adsabs.harvard.edu/abs/2017ApJ...846...99M/abstract)).

## Imports

- *s3fs* used to access cloud files as though they were local
- *astropy.units* used to define quantities with units attached
- *astropy.time Time* used to define the epoch for the ephemeris of the planet
- *astroquery.mast* used to search for and select data
- *lightkurve* for reading the light curve file
- *lcviz* for plotting and analyzing data

In [ ]:
import s3fs

import astropy.units as u
from astropy.time import Time
from astroquery.mast import Observations
import lightkurve

from lcviz import LCviz

***

## Using LCviz

### Initializing LCviz

Starting the LCviz application takes two steps: first, we need to create an instance of
the `LCviz` class, and then we need to show the application in the notebook. The two lines
below will open an empty instance of LCviz in the output cell of the notebook. 

The two main sections of LCviz that you will see below are a viewer for displaying flux vs time
data on the left, and an open tray with expandable sections giving UI access to data analysis plugins
on the right. Hovering the cursor over the various buttons in the application will display tooltips
giving a short description of their purpose.

In the next section, we will actually load data into LCviz to be shown in the flux vs time
viewer.

In [ ]:
# Create an instance of the LCviz class
lcviz = LCviz()

Now we show the application in the notebook. We keep this in a separate cell because
it is occasionally useful to refresh the display of the app without overwriting
it with a new instance:

In [ ]:
lcviz.show() # Show the LCviz app

It may be more convenient to open the application in a new tab within your Jupyter
Lab instance instead of scrolling up and down between the app and the code cells.
To do this you could un-comment and run the following line:

In [ ]:
# lcviz.show(loc="sidecar:tab-after") # Show the app in a new tab within Jupyter Lab

### Loading light curves into LCviz

Here we will use [astroquery](https://astroquery.readthedocs.io/en/latest/) to get the cloud location for the file that we want to load. For more information about the code below, see [this TIKE webinar notebook](../webinar-series/01-lightcurves/01-Lightcurves.ipynb).


In [ ]:
# enable searching on the cloud dataset
Observations.enable_cloud_dataset() 

With this feature enabled, we can search for our files. For this case, we'll use a known eclipsing binary from the Kepler missions.

In [ ]:
# query for our files of interest; in this case, Kepler observations of HAT-P-11
# to speed up our query, we'll use the kepler ID for HAT-P-11 (kplr010748390)
c_uris = Observations.get_cloud_uris(target_name="kplr010748390", 
                                    obs_collection="Kepler",
                                    dataproduct_type="timeseries",
                                    filter_products = {"productType": "SCIENCE", "productSubGroupDescription": "LLC"}
                                    )

# picking an arbitrary result; there are 15 to choose from
lc_uri = c_uris[8]

Now that we have the location for our file, we can load it into the lcviz app.

In [ ]:
# load the light curve into memory using lk.read
light_curve = lightkurve.read(lc_uri)
# load this data into lcviz
lcviz.load_data(light_curve)

## Flattening

There are some long term trends in the data; let's remove them so we can see the transits more clearly. Here we use `Flatten` plugin to removelow frequency trends from the light curves. Note that all of the options set here are also available in the UI in the `Flatten` plugin, accessible in the plugin tray on the right side of the app.

In [ ]:
flatten = lcviz.plugins['Flatten']

# There is currently only one dataset, so we do not
# need to set flatten.dataset. If there were multiple
# light curves loaded, you could see the choices to set
# this with flatten.dataset.choices
flatten.window_length = 50
flatten.flatten();

## Phase Folding

A common operation when dealing with light curves is to phase fold the data, so
that all of the transits are lined up with each other, essentially displaying all
of the data over a single orbit of the planet. Here we use the `Ephemeris` plugin
to phase fold the data using values from the literature for the epoch and period.
Running this code will create a second flux vs time viewer in the app to display
the new phase folded light curve.

In [ ]:
# get the origin of the time axis in LCviz:
data = lcviz.get_data()
reference_time = data.time[0]

# literature ephemeris for hot Neptune planet HAT-P-11 b:
morris2017_epoch = 2454605.89146  # BJD (TDB)
morris2017_period = 4.88780258  # days

# phase-fold the transit light curve in an ephemeris viewer:
eph = lcviz.plugins['Ephemeris']
eph.period = morris2017_period
eph.t0 = (
    (morris2017_epoch - reference_time.jd) % eph.period
)

# offset the wrapping phase so the transit (at phase 0) displays at center
eph.wrap_at = 0.5

## Additional Resources

- [LCviz documentation](https://lcviz.readthedocs.io/en/stable/index.html)
- [LCviz Github](https://github.com/spacetelescope/lcviz)
- [lightkurve documentation](https://lightkurve.github.io/lightkurve/)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)

## Citations

If you use `astropy`, `lightkurve`, or `Jdaviz` (via `LCviz`) for published research, please cite
the authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

And cite Jdaviz through its [Zenodo record](https://doi.org/10.5281/zenodo.5513927).


## About this Notebook

**Author(s):** Ricky O'Steen, Kyle Conroy, Brett Morris, Thomas Dutkiewicz <br/>
**Keyword(s):** Tutorial, LCviz, TESS, introduction <br/>
**Last updated:** Dec 2024 <br/>

***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/> 